In [1]:
import os
import numpy as np
from keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

model = load_model('glaucoma_detection_model.h5')

image_width = 224
image_height = 224

batch_size = 32

test_datagen = ImageDataGenerator(rescale=1.0/255)

positive_folder = 'Validation/Glaucoma_Positive'
negative_folder = 'Validation/Glaucoma_Negative/'

test_image_paths = []
test_labels = []

positive_image_paths = [os.path.join(positive_folder, filename) for filename in os.listdir(positive_folder)]
test_image_paths.extend(positive_image_paths)
test_labels.extend([1] * len(positive_image_paths))

negative_image_paths = [os.path.join(negative_folder, filename) for filename in os.listdir(negative_folder)]
test_image_paths.extend(negative_image_paths)
test_labels.extend([0] * len(negative_image_paths))

test_data = list(zip(test_image_paths, test_labels))

np.random.shuffle(test_data)

test_image_paths, test_labels = zip(*test_data)

test_image_paths = np.array(test_image_paths)
test_labels = np.array(test_labels)

test_labels = test_labels.astype(str)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': test_image_paths, 'class': test_labels}),
    x_col='filename',
    y_col='class',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False 
)

predictions = model.predict_generator(test_generator, steps=len(test_generator), verbose=1)
binary_predictions = (predictions > 0.5).astype(int)
accuracy = accuracy_score(test_labels.astype(int), binary_predictions)
f1 = f1_score(test_labels.astype(int), binary_predictions)
roc_auc = roc_auc_score(test_labels.astype(int), predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC: {roc_auc:.4f}")

2024-01-29 19:52:11.027082: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 19:52:11.066476: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 19:52:11.066574: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 19:52:11.067951: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 19:52:11.074328: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 19:52:11.074785: I tensorflow/core/platform/cpu_feature_guard.cc:1

Found 130 validated image filenames belonging to 2 classes.


/tmp/ipykernel_127759/2616042136.py:66: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(test_generator, steps=len(test_generator), verbose=1)


5/5 [==============================] - 10s 2s/step
Accuracy: 0.7385
F1 Score: 0.0000
AUC: 0.5000
